In [135]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [136]:
mutations_ARF9_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped.tsv'
mutations_ARF9_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_ARF9_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9/ARF9_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_ARF18_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped.tsv'
mutations_ARF18_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_ARF18_genotyped_only_mutated_filtered = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated_filtered.tsv'
mutations_ARF18_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'
mutations_ARF18_genotyped_only_mutated_flattened_filtered = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18_filtered/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_DREB26_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped.tsv'
mutations_DREB26_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_DREB26_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26/DREB26_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'

mutations_NLP7_genotyped = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped.tsv'
mutations_NLP7_genotyped_only_mutated = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped_only_mutated.tsv'
mutations_NLP7_genotyped_only_mutated_flattened = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7/NLP7_TFBSoverlapping_genotyped_only_mutated_flattened.tsv'


In [137]:
#read in files
def read_files(genotyped_file,genotyped_only_mutated_file,genotyped_only_mutated_flattened_file):
    """read in the files to pandas"""
    #read in files
    genotyped_df = pd.read_table(genotyped_file,sep='\t', header=0)
    genotyped_only_mutated_df = pd.read_table(genotyped_only_mutated_file,sep='\t', header=0)
    genotyped_only_mutated_flattened_df = pd.read_table(genotyped_only_mutated_flattened_file,sep='\t', header=0)
    return genotyped_df,genotyped_only_mutated_df,genotyped_only_mutated_flattened_df


In [138]:
def count_number_mutations_overlapping_at_least_one_TFBS(df, mutation_type):
    """function to count the number of mutations in the df which overlap at least one TFBS"""
    #turn df into a dict
    df_dict = df.to_dict(orient="records")
    mutations_overlapping_at_least_one_TFBS_count = 0
    if mutation_type == 'insertion':
        for row_dict in df_dict:
                list_lists_of_overlapping_AGIs = []
                genomic_positions = literal_eval(row_dict[f'{mutation_type}_genomic_positions_combined'])
                #iterate over insertions. Some insertions are linger than 1 bp so the genomic position pairs need combining
                #remaining pairs
                remaining_pairs = []
                for size in literal_eval(row_dict[f'{mutation_type}_sizes']):
                                
                    size = int(size)
                    #select pairs up to the size of the insertion
                    
                    #pop out the first n pairs from the genomic positions list, where n is the size of the insertion - 1
                    #if remaining pairs not empty, start with that
                    if len(remaining_pairs)>0:
                        #print('not []')
                        current_pairs = remaining_pairs[:size]
                    #else create the remaining pairs variable
                    elif len(remaining_pairs)==0:
                        #print('is []')
                        current_pairs = genomic_positions[:size]
                        remaining_pairs = genomic_positions[size:]          
                    #print(f'currentpairs={current_pairs}remaining={remaining_pairs}')

                    current_pairs_list_of_overlapping = []
                    #iterate over pairs in the mutation_genomic_positions_combined list #evaluate the string representing the pair as if they are literals ie. a list
                    for pair in current_pairs:
                        #print(f'pair={pair}')
                        list_of_AGIs = []
                        list_of_overlapping_AGIs = []
                        #for genomic position in pair 
                        #print(pair)
                        for pos in pair:
                            #make the genomic position string
                            pos = str(pos)
                            #if genomic position is in the mutation_overlapping_TFBS_AGI_combined dictionary
                            if pos in literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined']):
                                #print(pos)
                                #append to list_of_AGIs
                                mutation_overlapping_TFBS_AGI_combined = literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined'])
                            
                                #print(pos)
                                AGI = mutation_overlapping_TFBS_AGI_combined[pos]
                                list_of_AGIs += AGI
                        #if empty list, pass
                        
                        for elem in list_of_AGIs:
                            #print(elem)
                            #if TFBS is only overlapping one of the genomic positions in the pair, pass - we only want to include TFBSs that overlapping both positions so that the insertion is totally overlapping
                            if list_of_AGIs.count(elem) == 1:
                                pass
                            elif list_of_AGIs.count(elem) > 1:
                                list_of_overlapping_AGIs.append(elem)
                        #append to list of lists if lists_of_overlapping_AGIs is not empty
                        #remove duplicates from list_of_overlapping_AGIs
                        list_of_overlapping_AGIs_set = set(list_of_overlapping_AGIs)
                        list_of_overlapping_AGIs = list(list_of_overlapping_AGIs_set)
                        if len(list_of_overlapping_AGIs) > 0:
                            #print(list_of_overlapping_AGIs)
                            #get unique AGIs
                            #list_of_overlapping_AGIs_unique
                            current_pairs_list_of_overlapping+=list_of_overlapping_AGIs
                    #get unique elements
                    current_pairs_list_of_overlapping_set = set(current_pairs_list_of_overlapping)
                    current_pairs_list_of_overlapping = list(current_pairs_list_of_overlapping_set)
                    #if current_pairs_list_of_overlapping list isn't empty, append to list_lists_of_overlapping_AGIs 
                    if len(current_pairs_list_of_overlapping) > 0:
                        #print(current_pairs_list_of_overlapping)
                        #add one to the count of mutations which overlap at least one TFBS
                        mutations_overlapping_at_least_one_TFBS_count+=1
                        #append current_pairs_list_of_overlapping of TFBSs to list_lists_of_overlapping_AGIs
                        list_lists_of_overlapping_AGIs.append(current_pairs_list_of_overlapping)
    else:
        for row_dict in df_dict:
            list_lists_of_overlapping_AGIs = []
            genomic_positions = literal_eval(row_dict[f'{mutation_type}_genomic_positions_combined'])
            #iterate over insertions. Some insertions are linger than 1 bp so the genomic position pairs need combining
            #remaining positions
            remaining_positions = []
            for size in literal_eval(row_dict[f'{mutation_type}_sizes']):
                            
                size = int(size)
                #select positions up to the size of the insertion
                
                #pop out the first n positions from the genomic positions list, where n is the size of the insertion - 1
                #if remaining positions not empty, start with that
                if len(remaining_positions)>0:
                    current_positions = remaining_positions[:size]
                    #else create the remaining pairs variable
                elif len(remaining_positions)==0:
                    current_positions = genomic_positions[:size]
                    remaining_positions = genomic_positions[size:]
                list_of_overlapping_AGIs = []
                #for genomic position in pair 
                #print(pair)
                current_positions_list_of_overlapping = []
                for pos in current_positions:
                    #make the genomic position string
                    pos = str(pos)
                    #if genomic position is in the mutation_overlapping_TFBS_AGI_combined dictionary
                    if pos in literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined']):
                        #print(pos)
                        #append to list_of_overlapping_AGIs
                        mutation_overlapping_TFBS_AGI_combined = literal_eval(row_dict[f'{mutation_type}_overlapping_TFBS_AGI_combined'])
                    
                        #print(pos)
                        AGI = mutation_overlapping_TFBS_AGI_combined[pos]
                        list_of_overlapping_AGIs += AGI                     
                        #append to list of lists if lists_of_overlapping_AGIs is not empty
                #remove duplicates from list_of_overlapping_AGIs
                list_of_overlapping_AGIs_set = set(list_of_overlapping_AGIs)
                list_of_overlapping_AGIs = list(list_of_overlapping_AGIs_set)
                if len(list_of_overlapping_AGIs) > 0:
                    #print(list_of_overlapping_AGIs)
                    #get unique AGIs
                    #list_of_overlapping_AGIs_unique
                    current_positions_list_of_overlapping+=list_of_overlapping_AGIs
                #get unique elements
                current_positions_list_of_overlapping_set = set(current_positions_list_of_overlapping)
                current_positions_list_of_overlapping = list(current_positions_list_of_overlapping_set)
                #if current_positions_list_of_overlapping list isn't empty, append to list_lists_of_overlapping_AGIs 
                if len(current_positions_list_of_overlapping) > 0:
                    #add one to the count of mutations which overlap at least one TFBS
                    mutations_overlapping_at_least_one_TFBS_count+=1
                    #append current_positions_list_of_overlapping of TFBSs to list_lists_of_overlapping_AGIs
                    list_lists_of_overlapping_AGIs.append(current_positions_list_of_overlapping)

    return mutations_overlapping_at_least_one_TFBS_count
                

In [139]:
def count_mutations(genotyped_df,genotyped_only_mutated_df,genotyped_only_mutated_flattened_df):
    """function to count number of genotyped plant lines and also number of plant lines containing mutations, and then number of mutations overall"""
    #count number of unique plant IDs
    genotyped_counts = genotyped_df.plant_ID.unique()
    genotyped_only_mutated_counts = genotyped_only_mutated_df.plant_ID.unique()
    #print counts
    #total mutations
    #total_mutations = genotyped_only_mutated_flattened_df
    print(f'total plant lines sequenced={len(genotyped_counts)} \ntotal mutated plant lines={len(genotyped_only_mutated_counts)}')#\ntotal mutations={len(genotyped_only_mutated_df)}
    #count number of chimeric plant lines
    # unique_line_mutated = genotyped_only_mutated_df.drop_duplicates(subset='plant_ID',keep='first')
    
    unique_line_mutated = genotyped_only_mutated_flattened_df
    #filter so only unique plantids are present - drop duplicates
    #unique_unique_line_mutated = unique_line_mutated.drop_duplicates(subset='plant_ID',keep='first')

    non_chimeric = unique_line_mutated[unique_line_mutated.genotype != 'chimeric']
    non_chimeric_unique = non_chimeric.drop_duplicates(subset='plant_ID',keep='first')

    homozygous = unique_line_mutated[unique_line_mutated.genotype == 'homozygous']
    homozygous_unique = homozygous.drop_duplicates(subset='plant_ID',keep='first')

    heterozygous = unique_line_mutated[unique_line_mutated.genotype == 'heterozygous']
    heterozygous_unique = heterozygous.drop_duplicates(subset='plant_ID',keep='first')
    biallelic = unique_line_mutated[unique_line_mutated.genotype == 'biallelic']
    biallelic_unique = biallelic.drop_duplicates(subset='plant_ID',keep='first')

    #print bold heading
    print('\033[1m'+'Genotypes'+'\033[0m')
    print(f'number of non-chimeric mutated plant lines = {len(non_chimeric_unique)} ({len(non_chimeric_unique)/len(genotyped_only_mutated_counts)*100}% of non-chimeric lines)')
    print(f'number of homozygous mutated plant lines = {len(homozygous_unique)} ({len(homozygous_unique)/len(non_chimeric)*100}% of non-chimeric lines)')
    print(f'number of heterozygous mutated plant lines = {len(heterozygous_unique)} ({len(heterozygous_unique)/len(non_chimeric)*100}% of non-chimeric lines)')
    print(f'number of biallelic mutated plant lines = {len(biallelic_unique)} ({len(biallelic_unique)/len(non_chimeric)*100}% of non-chimeric lines)')
    #count number of non-chimeric insertions, deletions, substitutions (if two insertions around same guide, counted as once)
    non_chimeric_all = non_chimeric.copy()
    insertions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('insertion')]
    #print(insertions.insertion_sizes.apply(literal_eval).sum())
    deletions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('deletion')]
    substitutions = non_chimeric_all[non_chimeric_all.mutation_type.str.contains('substitution')]
    #number of insertions. evaluate the strings as if they are literals (as if they are lists)
    number_of_insertions = len(insertions.insertion_sizes.apply(literal_eval).sum())
    #print(np.concatenate(insertions.insertion_genomic_positions_combined)
    #number of deletions
    all_deletion_lengths = deletions.deletion_sizes.apply(literal_eval).sum()
    number_of_deletions = len(all_deletion_lengths)
    #number of substitutions
    number_of_substitutions = len(substitutions.substitution_sizes.apply(literal_eval).sum())
    #number of 1bp insertions
    insertions_1bp = list(filter(lambda x: x == 1, insertions.insertion_sizes.apply(literal_eval).sum()))
    insertions_percent_1bp = len(insertions_1bp)/number_of_insertions*100
    print(f'percentage insertions which are 1bp insertions = {insertions_percent_1bp}%')


    median_reads_guide_sites = genotyped_df.read_number.median()
    median_mutation_reads_guide_sites = genotyped_only_mutated_df.read_number.median()
    #print bold heading
    #filter out deletion values less than 10
    deletions_over_10bp = list(filter(lambda x: x > 10, all_deletion_lengths))
    total_mutations = number_of_insertions + number_of_deletions + number_of_substitutions
    print('\033[1m'+'Mutation types'+'\033[0m')
    print(f'total mutations = {total_mutations}')
    print(f'number of insertions = {number_of_insertions} ({number_of_insertions/total_mutations*100}% of total mutations)')
    print(f'number of deletions = {number_of_deletions} ({number_of_deletions/total_mutations*100}% of total mutations)')
    #deletions over 10bp
    #deletions.deletion_positions.str.len()
    #deletions_over_10bp = deletions[deletions.deletion_position.str.len()>10]
    
    #print(all_deletion_lengths)
    
    #print(deletions_over_10bp)
    print(f'deletions over 10bp = {len(deletions_over_10bp)} ({len(deletions_over_10bp)/number_of_deletions*100}% of deletions)')
    print(f'number of substitutions = {number_of_substitutions} ({number_of_substitutions/total_mutations*100}% of total mutations)')
    print(f'median reads at guide sites including wt = {median_reads_guide_sites}')
    print(f'median reads at guide sites mutations only = {median_mutation_reads_guide_sites}')
    
    #find number of mutations overlapping FIMO predicted TFBS
    insertions_overlapping_TFBS = insertions[insertions.insertion_overlapping_TFBS_AGI_combined.notna()]
    deletions_overlapping_TFBS = deletions[deletions.deletion_overlapping_TFBS_AGI_combined.notna()]
    substitutions_overlapping_TFBS = substitutions[substitutions.substitution_overlapping_TFBS_AGI_combined.notna()]
    #count number of insertions overlapping TFBSs
    insertions_count = count_number_mutations_overlapping_at_least_one_TFBS(insertions_overlapping_TFBS, 'insertion')
    deletions_count = count_number_mutations_overlapping_at_least_one_TFBS(deletions_overlapping_TFBS, 'deletion')
    substitutions_count = count_number_mutations_overlapping_at_least_one_TFBS(substitutions_overlapping_TFBS, 'substitution')
          

    #print bold heading
    print('\033[1m'+'Overlapping TFBSs'+'\033[0m')
    print(f'number of insertions overlapping TFBS = {insertions_count} ({insertions_count/number_of_insertions*100}% of insertions)')
    print(f'number of deletions overlapping TFBS = {deletions_count} ({deletions_count/number_of_deletions*100}% of insertions)')
    print(f'number of substitutions overlapping TFBS = {substitutions_count} ({substitutions_count/number_of_substitutions*100}% of insertions)')

    #how many within 7bp of cutsite
    #insertions_7bp = insertions[[[i for i in x if int(i) <8] for x in insertions.insertion_cut_site_distance.str.strip('][').str.split(', ')]]

   # print(insertions_7bp)


    return genotyped_counts, genotyped_only_mutated_counts, genotyped_only_mutated_df

In [140]:
ARF9_genotyped_df, ARF9_mutated_df,ARF9_flattened_df = read_files(mutations_ARF9_genotyped,mutations_ARF9_genotyped_only_mutated,mutations_ARF9_genotyped_only_mutated_flattened)


In [141]:
ARF9_genotyped_plant_lines, ARF9_mutated_plant_lines,ARF9_mutations = count_mutations(ARF9_genotyped_df,ARF9_mutated_df,ARF9_flattened_df)

total plant lines sequenced=468 
total mutated plant lines=147
Genotypes
number of non-chimeric mutated plant lines = 142 (96.5986394557823% of non-chimeric lines)
number of homozygous mutated plant lines = 92 (64.7887323943662% of non-chimeric lines)
number of heterozygous mutated plant lines = 12 (8.450704225352112% of non-chimeric lines)
number of biallelic mutated plant lines = 38 (26.76056338028169% of non-chimeric lines)
percentage insertions which are 1bp insertions = 98.35164835164835%
Mutation types
total mutations = 236
number of insertions = 182 (77.11864406779661% of total mutations)
number of deletions = 50 (21.1864406779661% of total mutations)
deletions over 10bp = 9 (18.0% of deletions)
number of substitutions = 4 (1.694915254237288% of total mutations)
median reads at guide sites including wt = 361.0
median reads at guide sites mutations only = 234.0
Overlapping TFBSs
number of insertions overlapping TFBS = 34 (18.681318681318682% of insertions)
number of deletions ove

In [142]:
#filter ARF18 guide 14 since very repetitive and lots of sequencing errors
ARF18_mutated_df = pd.read_table(mutations_ARF18_genotyped_only_mutated,sep='\t', header=0)
ARF18_mutated_plant_lines_filtered = ARF18_mutated_df[ARF18_mutated_df.guide != 'ARF18_guide14']
#save df
ARF18_mutated_plant_lines_filtered.to_csv(mutations_ARF18_genotyped_only_mutated_filtered, sep='\t',header=True)
#Then I ran pacbio_analyse_flattened_mutations.py on the filtered .tsv to generate the mutations_ARF18_genotyped_only_mutated_flattened_filtered file (f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18_filtered/ARF18_TFBSoverlapping_genotyped_only_mutated_flattened.tsv')


In [143]:
ARF18_genotyped_df, ARF18_mutated_df,ARF18_flattened_filtered_df = read_files(mutations_ARF18_genotyped,mutations_ARF18_genotyped_only_mutated,mutations_ARF18_genotyped_only_mutated_flattened_filtered)

#filter ARF18 guide 14 since very repetitive and lots of sequencing errors
ARF18_genotyped_only_mutated_df = pd.read_table(mutations_ARF18_genotyped_only_mutated,sep='\t', header=0)
ARF18_mutated_plant_lines_filtered = ARF18_mutated_df[ARF18_mutated_df.guide != 'ARF18_guide14']
#save as df

In [144]:

#count mutations
ARF18_genotyped_plant_lines, ARF18_mutated_plant_lines,ARF18_mutations = count_mutations(ARF18_genotyped_df,ARF18_mutated_plant_lines_filtered,ARF18_flattened_filtered_df)

total plant lines sequenced=373 
total mutated plant lines=67
Genotypes
number of non-chimeric mutated plant lines = 66 (98.50746268656717% of non-chimeric lines)
number of homozygous mutated plant lines = 27 (40.909090909090914% of non-chimeric lines)
number of heterozygous mutated plant lines = 25 (37.878787878787875% of non-chimeric lines)
number of biallelic mutated plant lines = 14 (21.21212121212121% of non-chimeric lines)
percentage insertions which are 1bp insertions = 100.0%
Mutation types
total mutations = 106
number of insertions = 87 (82.0754716981132% of total mutations)
number of deletions = 11 (10.377358490566039% of total mutations)
deletions over 10bp = 3 (27.27272727272727% of deletions)
number of substitutions = 8 (7.547169811320755% of total mutations)
median reads at guide sites including wt = 623.0
median reads at guide sites mutations only = 310.5
Overlapping TFBSs
number of insertions overlapping TFBS = 53 (60.91954022988506% of insertions)
number of deletions o

In [145]:
DREB26_genotyped_df, DREB26_mutated_df,DREB26_flattened_df = read_files(mutations_DREB26_genotyped,mutations_DREB26_genotyped_only_mutated,mutations_DREB26_genotyped_only_mutated_flattened)
DREB26_genotyped_plant_lines, DREB26_mutated_plant_lines,DREB26_mutations = count_mutations(DREB26_genotyped_df,DREB26_mutated_df,DREB26_flattened_df)

total plant lines sequenced=366 
total mutated plant lines=43
Genotypes
number of non-chimeric mutated plant lines = 40 (93.02325581395348% of non-chimeric lines)
number of homozygous mutated plant lines = 15 (37.5% of non-chimeric lines)
number of heterozygous mutated plant lines = 19 (47.5% of non-chimeric lines)
number of biallelic mutated plant lines = 6 (15.0% of non-chimeric lines)
percentage insertions which are 1bp insertions = 100.0%
Mutation types
total mutations = 73
number of insertions = 40 (54.794520547945204% of total mutations)
number of deletions = 22 (30.136986301369863% of total mutations)
deletions over 10bp = 2 (9.090909090909092% of deletions)
number of substitutions = 11 (15.068493150684931% of total mutations)
median reads at guide sites including wt = 344.0
median reads at guide sites mutations only = 113.0
Overlapping TFBSs
number of insertions overlapping TFBS = 18 (45.0% of insertions)
number of deletions overlapping TFBS = 16 (72.72727272727273% of insertio

In [146]:
NLP7_genotyped_df, NLP7_mutated_df,NLP7_flattened_df = read_files(mutations_NLP7_genotyped,mutations_NLP7_genotyped_only_mutated,mutations_NLP7_genotyped_only_mutated_flattened)
NLP7_genotyped_plant_lines, NLP7_mutated_plant_lines,NLP7_mutations = count_mutations(NLP7_genotyped_df,NLP7_mutated_df,NLP7_flattened_df)

total plant lines sequenced=478 
total mutated plant lines=112
Genotypes
number of non-chimeric mutated plant lines = 107 (95.53571428571429% of non-chimeric lines)
number of homozygous mutated plant lines = 53 (49.532710280373834% of non-chimeric lines)
number of heterozygous mutated plant lines = 32 (29.906542056074763% of non-chimeric lines)
number of biallelic mutated plant lines = 22 (20.5607476635514% of non-chimeric lines)
percentage insertions which are 1bp insertions = 100.0%
Mutation types
total mutations = 180
number of insertions = 127 (70.55555555555556% of total mutations)
number of deletions = 39 (21.666666666666668% of total mutations)
deletions over 10bp = 10 (25.64102564102564% of deletions)
number of substitutions = 14 (7.777777777777778% of total mutations)
median reads at guide sites including wt = 646.0
median reads at guide sites mutations only = 349.0
Overlapping TFBSs
number of insertions overlapping TFBS = 41 (32.28346456692913% of insertions)
number of deleti

In [147]:
# make a combined dataframe of all the mutations
genotyped_df_all = pd.concat([ARF9_genotyped_df,ARF18_genotyped_df,DREB26_genotyped_df,NLP7_genotyped_df])
genotyped_only_mutated_df_all = pd.concat([ARF9_mutated_df,ARF18_mutated_plant_lines_filtered,DREB26_mutated_df,NLP7_mutated_df])
genotyped_only_mutated_flattened_df_all = pd.concat([ARF9_flattened_df,ARF18_flattened_filtered_df,DREB26_flattened_df,NLP7_flattened_df])


In [148]:
genotyped_only_mutated_flattened_df_all

,plant_ID,chr,platename,library,first_reaction_primers,second_reaction_primers,genotype,mutation_type,guides,mutation_genotypes,...,deletion_genomic_positions_combined,deletion_positions_relative_to_TSS_combined,deletion_sizes,deletion_overlapping_TFBS_AGI_combined,deletion_overlapping_TFBS_family_combined,substitution_genomic_positions_combined,substitution_positions_relative_to_TSS_combined,substitution_sizes,substitution_overlapping_TFBS_AGI_combined,substitution_overlapping_TFBS_family_combined
0,plntEPSWT20056-1,4,p6ARF9,1022,SW348_SW444,ARF9_4F_6R,biallelic,['deletion'],"['ARF9_guide1', 'ARF9_guide2', 'ARF9_guide2', ...","['biallelic', 'heterozygous']",...,"[12451335, 12451336, 12451337, 12451338, 12451...","[193, 194, 195, 196, 197, 198, 199]",[7],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plntEPSWT20056-2,4,p6ARF9,1022,SW360_SW456,ARF9_4F_6R,biallelic,['deletion'],"['ARF9_guide2', 'ARF9_guide3', 'ARF9_guide3']","['biallelic', 'homozygous']",...,"[12451334, 12451335, 12451336, 12451337]","[192, 193, 194, 195]",[4],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plntEPSWT20056-3,4,p6ARF9,1022,SW372_SW468,ARF9_4F_6R,biallelic,['deletion'],"['ARF9_guide2', 'ARF9_guide2', 'ARF9_guide3', ...","['biallelic', 'heterozygous']",...,"[12451332, 12451333, 12451334, 12451335, 12451...","[190, 191, 192, 193, 194, 195, 196, 197]",[8],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,plntEPSWT20056-4,4,p6ARF9,1022,SW384_SW480,ARF9_4F_6R,biallelic,"['deletion', 'insertion']","['ARF9_guide1', 'ARF9_guide2', 'ARF9_guide2', ...","['biallelic', 'heterozygous']",...,"[12451338, 12451339, 12451340, 12451341]","[196, 197, 198, 199]",[4],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,plntEPSWT20056-5,4,p6ARF9,1022,SW396_SW492,ARF9_4F_6R,chimeric,['deletion'],"['ARF9_guide1', 'ARF9_guide1', 'ARF9_guide2', ...","['biallelic', 'chimeric']",...,"[12451333, 12451334, 12451335, 12451336, 12451...","[191, 192, 193, 194, 195, 196, 197, 198, 199]",[9],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,plntEPSWT20197-6,4,p5NLP7,1021,SW390_SW486,NLP7_4F_6R,homozygous,['insertion'],['NLP7_guide23'],['homozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,plntEPSWT20197-7,4,p5NLP7,1021,SW402_SW498,NLP7_4F_6R,heterozygous,['deletion'],"['NLP7_guide23', 'NLP7_guide24']",['heterozygous'],...,"[12479206, 12479207, 12479208, 12479209, 12479...","[-313, -314, -315, -316, -317, -318, -319, -32...",[9],NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,plntEPSWT20197-8,4,p5NLP7,1021,SW414_SW510,NLP7_4F_6R,heterozygous,['insertion'],['NLP7_guide23'],['heterozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,plntEPSWT20197-10,4,p5NLP7,1021,SW438_SW534,NLP7_4F_6R,heterozygous,['insertion'],"['NLP7_guide18', 'NLP7_guide19', 'NLP7_guide20...","['heterozygous', 'homozygous']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
#find plant lines with mutations in more than one gene
genotyped_only_mutated_flattened_df_all['gene'] = genotyped_only_mutated_flattened_df_all['second_reaction_primers'].str.split('_').str[0]
print(genotyped_only_mutated_flattened_df_all.gene.unique())
unique_mutated_lines = genotyped_only_mutated_flattened_df_all['plant_ID'].unique()
print(f'unique mutated plant lines = {len(unique_mutated_lines)}')
# make df containing only plant lines with mutations in more than one gene
#make df containing all plant lines with duplicates based on the plant_ID column
duplicated_plant_lines_all = genotyped_only_mutated_flattened_df_all[genotyped_only_mutated_flattened_df_all.duplicated(subset=['plant_ID'], keep=False)]
#get unique plant lines
duplicated_plant_lines_unique = duplicated_plant_lines_all['plant_ID'].unique()
# duplicated_plant_lines_unique = duplicated_plant_lines_all[]
#sort by plant_id
duplicated_plant_lines_all = duplicated_plant_lines_all.sort_values(by=['plant_ID'])
# duplicated_plant_lines_unique = duplicated_plant_lines_unique.sort_values(by=['plant_ID'])
#reset index
duplicated_plant_lines_all = duplicated_plant_lines_all.reset_index(drop=True)
# duplicated_plant_lines_unique = duplicated_plant_lines_unique.reset_index(drop=True)

#remove nan values
# duplicated_plant_lines_unique = duplicated_plant_lines_unique[duplicated_plant_lines_unique['plant_ID'].notna()]
print(f'number of plant lines with mutations in more than one gene = {len(duplicated_plant_lines_unique)}')
#duplicated_plant_lines_all = genotyped_only_mutated_flattened_df_all[genotyped_only_mutated_flattened_df_all.duplicated(subset=['plant_ID'], keep=True)]
#make new flattened df containing list of unique plants with a column showing which genes they have mutations in
# flattened_multiple_mutations = duplicated_plant_lines_all.pivot_table(index='plant_ID', columns='gene', values='gene', aggfunc='count')
# # print(len(flattened_multiple_mutations)) 
# # print(flattened_multiple_mutations)
# print(len(flattened_multiple_mutations))
# print(flattened_multiple_mutations)
# create flattened_multiple_mutations containing only unique plant lines, with a gene column containing a list of genes with mutations
#flattened_multiple_mutations = duplicated_plant_lines_unique['plant_ID'].to_frame()

flattened_multiple_mutations = duplicated_plant_lines_all.groupby('plant_ID')['gene'].apply(list).to_frame()
flattened_multiple_mutations = flattened_multiple_mutations.reset_index()
flattened_multiple_mutations = flattened_multiple_mutations.rename(columns={'gene':'genes_with_mutations'})
#make into df
flattened_multiple_mutations = pd.DataFrame(flattened_multiple_mutations)
#sort by plant_id
flattened_multiple_mutations = flattened_multiple_mutations.sort_values(by=['plant_ID'])
print(len(flattened_multiple_mutations))
flattened_multiple_mutations
# #check differences in plant_ID column between flattened_multiple_mutations and duplicated_plant_lines_unique
# print(len(flattened_multiple_mutations['plant_ID'].unique()))
# print(len(duplicated_plant_lines_unique['plant_ID'].unique()))
# print(flattened_multiple_mutations['plant_ID'].unique() == duplicated_plant_lines_unique['plant_ID'].unique())
# print(flattened_multiple_mutations['plant_ID'].unique() != duplicated_plant_lines_unique['plant_ID'].unique())
#count how many genes_with_mutations lists are unique
# out = flattened_multiple_mutations.genes_with_mutations.apply(tuple).drop_duplicates().apply(list).tolist()
flattened_multiple_mutations['genes_with_mutations_sorted'] = flattened_multiple_mutations.genes_with_mutations.sort_values().apply(lambda x: sorted(x))
print(flattened_multiple_mutations['genes_with_mutations_sorted'].apply(tuple).value_counts())


['ARF9' 'ARF18' 'DREB26' 'NLP7']
unique mutated plant lines = 327
number of plant lines with mutations in more than one gene = 41
41
(ARF9, NLP7)            27
(ARF18, ARF9)            9
(ARF18, NLP7)            2
(ARF9, DREB26)           2
(ARF9, DREB26, NLP7)     1
Name: genes_with_mutations_sorted, dtype: int64


In [157]:
duplicated_plant_lines_unique

,plant_ID,chr,platename,library,first_reaction_primers,second_reaction_primers,genotype,mutation_type,guides,mutation_genotypes,...,deletion_positions_relative_to_TSS_combined,deletion_sizes,deletion_overlapping_TFBS_AGI_combined,deletion_overlapping_TFBS_family_combined,substitution_genomic_positions_combined,substitution_positions_relative_to_TSS_combined,substitution_sizes,substitution_overlapping_TFBS_AGI_combined,substitution_overlapping_TFBS_family_combined,gene
0,plntEPSWT20056-1,4,p6NLP7,1022,SW348_SW444,NLP7_4F_6R,biallelic,"['deletion', 'insertion']","['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...","['biallelic', 'heterozygous', 'homozygous']",...,"[124, 125, 126, 127, 128, 129, 130, 131, 132, ...",[34],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
1,plntEPSWT20056-10,4,p6NLP7,1022,SW361_SW457,NLP7_4F_6R,homozygous,['insertion'],"['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...",['homozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
2,plntEPSWT20056-2,4,p6NLP7,1022,SW360_SW456,NLP7_4F_6R,homozygous,['insertion'],"['NLP7_guide24', 'NLP7_guide25']",['homozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
3,plntEPSWT20056-3,4,p6NLP7,1022,SW372_SW468,NLP7_4F_6R,biallelic,['insertion'],"['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...","['biallelic', 'homozygous']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
4,plntEPSWT20056-5,4,p6NLP7,1022,SW396_SW492,NLP7_4F_6R,biallelic,"['deletion', 'insertion']","['NLP7_guide5', 'NLP7_guide5', 'NLP7_guide6', ...","['biallelic', 'heterozygous']",...,"[124, 125, 126, 127, 128, 129, 130, 131, 132, ...",[34],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
5,plntEPSWT20056-7,4,p6NLP7,1022,SW420_SW516,NLP7_4F_6R,homozygous,['insertion'],"['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...",['homozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
6,plntEPSWT20056-8,4,p6NLP7,1022,SW432_SW528,NLP7_4F_6R,heterozygous,['insertion'],"['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...",['heterozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
7,plntEPSWT20056-9,4,p6NLP7,1022,SW349_SW445,NLP7_4F_6R,heterozygous,['insertion'],"['NLP7_guide5', 'NLP7_guide6', 'NLP7_guide10',...","['heterozygous', 'homozygous']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7
8,plntEPSWT20072-9,4,p2NLP7,1018,SW387_SW483,NLP7_4F_6R,biallelic,"['insertion', 'substitution']","['NLP7_guide18', 'NLP7_guide18', 'NLP7_guide19...",['biallelic'],...,NaN,NaN,NaN,NaN,[12479337],[-190],[1],NaN,NaN,NLP7
9,plntEPSWT20075-1,4,p2NLP7,1018,SW411_SW507,NLP7_4F_6R,homozygous,['insertion'],"['NLP7_guide29', 'NLP7_guide30']",['homozygous'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NLP7


In [150]:
count_mutations(genotyped_df_all,genotyped_only_mutated_df_all,genotyped_only_mutated_flattened_df_all)

total plant lines sequenced=499 
total mutated plant lines=327
Genotypes
number of non-chimeric mutated plant lines = 315 (96.3302752293578% of non-chimeric lines)
number of homozygous mutated plant lines = 176 (49.57746478873239% of non-chimeric lines)
number of heterozygous mutated plant lines = 84 (23.661971830985916% of non-chimeric lines)
number of biallelic mutated plant lines = 73 (20.56338028169014% of non-chimeric lines)
percentage insertions which are 1bp insertions = 99.31192660550458%
Mutation types
total mutations = 595
number of insertions = 436 (73.27731092436974% of total mutations)
number of deletions = 122 (20.504201680672267% of total mutations)
deletions over 10bp = 24 (19.672131147540984% of deletions)
number of substitutions = 37 (6.218487394957983% of total mutations)
median reads at guide sites including wt = 475.0
median reads at guide sites mutations only = 253.0
Overlapping TFBSs
number of insertions overlapping TFBS = 146 (33.48623853211009% of insertions)
n

(array(['plntEPSWT20057-1', 'plntEPSWT20057-10', 'plntEPSWT20057-2',
        'plntEPSWT20057-3', 'plntEPSWT20057-4', 'plntEPSWT20057-5',
        'plntEPSWT20057-6', 'plntEPSWT20057-7', 'plntEPSWT20057-8',
        'plntEPSWT20057-9', 'plntEPSWT20058-1', 'plntEPSWT20058-10',
        'plntEPSWT20058-2', 'plntEPSWT20058-3', 'plntEPSWT20058-5',
        'plntEPSWT20058-6', 'plntEPSWT20058-7', 'plntEPSWT20058-8',
        'plntEPSWT20058-9', 'plntEPSWT20059-1', 'plntEPSWT20059-10',
        'plntEPSWT20059-2', 'plntEPSWT20059-3', 'plntEPSWT20059-4',
        'plntEPSWT20059-5', 'plntEPSWT20059-6', 'plntEPSWT20059-7',
        'plntEPSWT20059-8', 'plntEPSWT20059-9', 'plntEPSWT20061-1',
        'plntEPSWT20061-10', 'plntEPSWT20061-2', 'plntEPSWT20061-3',
        'plntEPSWT20061-4', 'plntEPSWT20061-5', 'plntEPSWT20061-6',
        'plntEPSWT20061-7', 'plntEPSWT20061-8', 'plntEPSWT20061-9',
        'plntEPSWT20062-1', 'plntEPSWT20062-10', 'plntEPSWT20062-2',
        'plntEPSWT20062-3', 'plntEPSWT20062